In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [482]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

In [3]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2010-2019_fl.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alachua,252475,255456,259052,263959,266309,268851,269043
1,Baker,27009,27122,27355,27884,28254,28353,29210
2,Bay,174596,178289,181488,183634,184736,186240,174705
3,Bradford,26802,26552,26748,26740,27142,27752,28201
4,Brevard,550255,555548,565746,576874,587769,595203,601942


In [4]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2020-2021_fl.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Alachua,278684,279238
1,Baker,28309,28715
2,Bay,174461,179168
3,Bradford,28364,28540
4,Brevard,608007,616628


In [5]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alachua,252475,255456,259052,263959,266309,268851,269043,278684,279238
1,Baker,27009,27122,27355,27884,28254,28353,29210,28309,28715
2,Bay,174596,178289,181488,183634,184736,186240,174705,174461,179168
3,Bradford,26802,26552,26748,26740,27142,27752,28201,28364,28540
4,Brevard,550255,555548,565746,576874,587769,595203,601942,608007,616628


In [6]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_fl = metros_df[metros_df['State Name']=='Florida']
metros_fl.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
0,"Gainesville, FL",Metropolitan Statistical Area,Alachua,Florida
14,"Jacksonville, FL",Metropolitan Statistical Area,Baker,Florida
20,"Panama City, FL",Metropolitan Statistical Area,Bay,Florida
34,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area,Brevard,Florida
36,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area,Broward,Florida


### Extracting data for year 2013 and combining with pop along with Metropolitans and Micropolitans

In [315]:
file_2013 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2013.pdf'

In [316]:
fl_df_crashes13 = pd.DataFrame()
pages = list(range(43,45))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_crashes13 = fl_df_crashes13.append(df)

In [317]:
fl_df_crashes13 = fl_df_crashes13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_crashes13 = fl_df_crashes13.iloc[:-2 , :]
fl_df_crashes13 = fl_df_crashes13.iloc[2: , :]
fl_df_crashes13.columns = ['County','total_crashes']
fl_df_crashes13.head()

,County,total_crashes
2,Alachua,"4,651"
3,Baker,293
4,Bay,"3,514"
5,Bradford,220
6,Brevard,"7,079"


In [318]:
fl_df_crashes13.tail()

,County,total_crashes
26,Unknown,0
27,Volusia,"8,047"
28,Wakulla,248
29,Walton,"1,088"
30,Washington,288


In [319]:
fl_df_fatalaties13 = pd.DataFrame()
pages = list(range(45,47))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_fatalaties13 = fl_df_fatalaties13.append(df)

In [320]:
fl_df_fatalaties13 = fl_df_fatalaties13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[:-2 , :]
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[2: , :]
fl_df_fatalaties13.columns = ['County','fatalities']
fl_df_fatalaties13.head()

,County,fatalities
2,Alachua,33
3,Baker,9
4,Bay,17
5,Bradford,5
6,Brevard,76


In [321]:
fl_df_fatalaties13.tail()

,County,fatalities
25,Unknown,0
26,Volusia,90
27,Wakulla,3
28,Walton,17
29,Washington,8


In [322]:
fl_df_injuries13 = pd.DataFrame()
pages = list(range(47,49))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_injuries13 = fl_df_injuries13.append(df)

In [323]:
fl_df_injuries13 = fl_df_injuries13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_injuries13 = fl_df_injuries13.iloc[:-2 , :]
fl_df_injuries13 = fl_df_injuries13.iloc[2: , :]
fl_df_injuries13.columns = ['County','injuries']
fl_df_injuries13.head()

,County,injuries
2,Alachua,"3,321"
3,Baker,270
4,Bay,"2,208"
5,Bradford,202
6,Brevard,"5,041"


In [324]:
fl_df_injuries13.tail()

,County,injuries
25,Unknown,0
26,Volusia,"5,210"
27,Wakulla,181
28,Walton,720
29,Washington,230


In [334]:
fl_df_2013 = pd.merge(pd.merge(fl_df_fatalaties13,fl_df_injuries13,on='County'),fl_df_crashes13,on='County')
fl_df_2013 = fl_df_2013.drop(labels=63, axis=0)
fl_df_2013.head(10)

,County,fatalities,injuries,total_crashes
0,Alachua,33,"3,321","4,651"
1,Baker,9,270,293
2,Bay,17,"2,208","3,514"
3,Bradford,5,202,220
4,Brevard,76,"5,041","7,079"
5,Broward,180,"21,580","32,595"
6,Calhoun,1,109,152
7,Charlotte,22,"1,412","2,091"
8,Citrus,17,993,"1,360"
9,Clay,10,"1,621","2,434"


In [335]:
fl_df_2013.tail(10)

,County,fatalities,injuries,total_crashes
57,St. Johns,31,"2,032","2,730"
58,St. Lucie,33,"1,864","2,931"
59,Sumter,19,529,724
60,Suwannee,13,337,526
61,Taylor,6,149,198
62,Union,2,95,125
64,Volusia,90,"5,210","8,047"
65,Wakulla,3,181,248
66,Walton,17,720,"1,088"
67,Washington,8,230,288


In [336]:
df_combined_fl_13 = pd.merge(fl_df_2013,pop_combined[['County', 2013]],on="County",how='left')
df_combined_fl_2013 = pd.merge(df_combined_fl_13,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2013.head(10)

,County,fatalities,injuries,total_crashes,2013,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,33,"3,321","4,651",252475.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,9,270,293,27009.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,17,"2,208","3,514",174596.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,5,202,220,26802.0,NaN,NaN
4,Brevard,76,"5,041","7,079",550255.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area
5,Broward,180,"21,580","32,595",1836600.0,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area
6,Calhoun,1,109,152,14578.0,NaN,NaN
7,Charlotte,22,"1,412","2,091",164671.0,"Punta Gorda, FL",Metropolitan Statistical Area
8,Citrus,17,993,"1,360",138829.0,"Homosassa Springs, FL",Metropolitan Statistical Area
9,Clay,10,"1,621","2,434",195657.0,"Jacksonville, FL",Metropolitan Statistical Area


In [337]:
df_combined_fl_2013.tail(10)

,County,fatalities,injuries,total_crashes,2013,CBSA Title,Metropolitan/Micropolitan Statistical Area
57,St. Johns,31,"2,032","2,730",NaN,"Jacksonville, FL",Metropolitan Statistical Area
58,St. Lucie,33,"1,864","2,931",NaN,"Port St. Lucie, FL",Metropolitan Statistical Area
59,Sumter,19,529,724,106898.0,"The Villages, FL",Metropolitan Statistical Area
60,Suwannee,13,337,526,43588.0,NaN,NaN
61,Taylor,6,149,198,22873.0,NaN,NaN
62,Union,2,95,125,15160.0,NaN,NaN
63,Volusia,90,"5,210","8,047",500438.0,"Deltona-Daytona Beach-Ormond Beach, FL",Metropolitan Statistical Area
64,Wakulla,3,181,248,31004.0,"Tallahassee, FL",Metropolitan Statistical Area
65,Walton,17,720,"1,088",59193.0,"Crestview-Fort Walton Beach-Destin, FL",Metropolitan Statistical Area
66,Washington,8,230,288,24504.0,NaN,NaN


In [338]:
df_combined_fl_2013.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2013_FL.xlsx', engine='xlsxwriter',index=False)

In [12]:
file_2014_2016 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2016.pdf'

In [13]:
pdf = pdfplumber.open(file_2014_2016)
table=pdf.pages[35].extract_table()
table = table[1:]
table

[['Alachua', 'Total Crashes', '5,980', '5,027', '4,193', '18.96%', '5,067'],
 [None, 'Total Fatalities', '46', '37', '30', '24.32%', '38'],
 [None, 'Total Injuries', '4,398', '3,826', '3,113', '14.95%', '3,779'],
 [None, 'Total', '10,424.00', '8,890.00', '7,336.00', '17.26%', '8,883.00'],
 ['Baker', 'Total Crashes', '381', '373', '246', '2.14%', '333'],
 [None, 'Total Fatalities', '13', '12', '2', '8.33%', '9'],
 [None, 'Total Injuries', '352', '286', '264', '23.08%', '301'],
 [None, 'Total', '746.00', '671.00', '512.00', '11.18%', '643.00'],
 ['Bay', 'Total Crashes', '4,089', '3,946', '3,337', '3.62%', '3,791'],
 [None, 'Total Fatalities', '38', '32', '30', '18.75%', '33'],
 [None, 'Total Injuries', '2,290', '2,317', '2,023', '-1.17%', '2,210'],
 [None, 'Total', '6,417.00', '6,295.00', '5,390.00', '1.94%', '6,034.00'],
 ['Bradford', 'Total Crashes', '362', '348', '274', '4.02%', '328'],
 [None, 'Total Fatalities', '6', '10', '4', '-40.00%', '7'],
 [None, 'Total Injuries', '243', '241'

In [14]:
pdf = pdfplumber.open(file_2014_2016)
table=pdf.pages[37].extract_table()
table = table[1:]
list_length = len(table)
paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
if list_length % 3 == 2:
   paired_list.append(table[list_length-1])
print(paired_list)    

[['Gilchrist', 'Total Crashes', '194', '176', '160', '10.23%', '177', None, 'Total Fatalities', '5', '3', '3', '66.67%', '4', None, 'Total Injuries', '123', '126', '109', '-2.38%', '119'], ['Glades', 'Total Crashes', '178', '155', '139', '14.84%', '157', None, 'Total Fatalities', '2', '14', '3', '-85.71%', '6', None, 'Total Injuries', '97', '114', '96', '-14.91%', '102'], ['Gulf', 'Total Crashes', '181', '140', '137', '29.29%', '153', None, 'Total Fatalities', '1', '2', '2', '-50.00%', '2', None, 'Total Injuries', '119', '81', '98', '46.91%', '99'], ['Hamilton', 'Total Crashes', '269', '253', '177', '6.32%', '233', None, 'Total Fatalities', '5', '5', '6', '0.00%', '5', None, 'Total Injuries', '159', '151', '129', '5.30%', '146'], ['Hardee', 'Total Crashes', '416', '382', '333', '8.90%', '377', None, 'Total Fatalities', '5', '2', '10', '150.00%', '6', None, 'Total Injuries', '334', '292', '256', '14.38%', '294'], ['Hendry', 'Total Crashes', '331', '270', '254', '22.59%', '285', None, 'T

In [15]:
df_test = pd.DataFrame(paired_list)
df_test = df_test.drop([5,6,7,12,13,14,19,20],axis=1)
df_test

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Gilchrist,Total Crashes,194,176,160,Total Fatalities,5,3,3,Total Injuries,123,126,109
1,Glades,Total Crashes,178,155,139,Total Fatalities,2,14,3,Total Injuries,97,114,96
2,Gulf,Total Crashes,181,140,137,Total Fatalities,1,2,2,Total Injuries,119,81,98
3,Hamilton,Total Crashes,269,253,177,Total Fatalities,5,5,6,Total Injuries,159,151,129
4,Hardee,Total Crashes,416,382,333,Total Fatalities,5,2,10,Total Injuries,334,292,256
5,Hendry,Total Crashes,331,270,254,Total Fatalities,19,10,8,Total Injuries,273,235,228
6,Hernando,Total Crashes,"2,142","2,153","1,981",Total Fatalities,25,35,20,Total Injuries,"2,093","2,052","1,764"
7,Highlands,Total Crashes,"1,329","1,218","1,067",Total Fatalities,21,22,21,Total Injuries,978,836,740
8,Hillsborough,Total Crashes,"27,660","25,967","23,024",Total Fatalities,228,190,158,Total Injuries,"21,380","19,889","17,561"
9,Holmes,Total Crashes,284,259,222,Total Fatalities,9,9,5,Total Injuries,158,141,169


In [16]:
df_test_melt_2016 = pd.melt(df_test,id_vars=[2,9,16],value_vars=[0])
df_test_melt_2016 = df_test_melt_2016.drop(['variable'],axis=1)
df_test_melt_2016.head()

,2,9,16,value
0,194,5,123,Gilchrist
1,178,2,97,Glades
2,181,1,119,Gulf
3,269,5,159,Hamilton
4,416,5,334,Hardee


In [17]:
df_test_melt_2015 = pd.melt(df_test,id_vars=[3,10,17],value_vars=[0])
df_test_melt_2015 = df_test_melt_2015.drop(['variable'],axis=1)
df_test_melt_2015.head()

,3,10,17,value
0,176,3,126,Gilchrist
1,155,14,114,Glades
2,140,2,81,Gulf
3,253,5,151,Hamilton
4,382,2,292,Hardee


In [18]:
df_test_melt_2014 = pd.melt(df_test,id_vars=[4,11,18],value_vars=[0])
df_test_melt_2014 = df_test_melt_2014.drop(['variable'],axis=1)
df_test_melt_2014.head()

,4,11,18,value
0,160,3,109,Gilchrist
1,139,3,96,Glades
2,137,2,98,Gulf
3,177,6,129,Hamilton
4,333,10,256,Hardee


### Extracting data for years 2014 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [19]:
fl_df_2014_2016 = pd.DataFrame()
pages = list(range(35,42))
pdf = pdfplumber.open(file_2014_2016)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2014_2016 = fl_df_2014_2016.append(df)

In [20]:
fl_df_2014_2016 = fl_df_2014_2016.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2014_2016.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"5,980","5,027","4,193",Total Fatalities,46,37,30,Total Injuries,"4,398","3,826","3,113"
1,Baker,Total Crashes,381,373,246,Total Fatalities,13,12,2,Total Injuries,352,286,264
2,Bay,Total Crashes,"4,089","3,946","3,337",Total Fatalities,38,32,30,Total Injuries,"2,290","2,317","2,023"
3,Bradford,Total Crashes,362,348,274,Total Fatalities,6,10,4,Total Injuries,243,241,219
4,Brevard,Total Crashes,"8,752","8,387","7,661",Total Fatalities,98,82,65,Total Injuries,"6,564","6,130","5,386"


In [49]:
fl_df_2014_2016.tail()

,0,1,2,3,4,8,9,10,11,15,16,17,18
10,Union,Total Crashes,122,103,114,Total Fatalities,2,3,1,Total Injuries,84,110,131
0,Volusia,Total Crashes,"8,903","8,851","8,382",Total Fatalities,122,87,86,Total Injuries,"5,872","5,750","5,251"
1,Wakulla,Total Crashes,366,348,317,Total Fatalities,14,2,2,Total Injuries,264,261,231
2,Walton,Total Crashes,"1,305","1,248","1,127",Total Fatalities,17,9,13,Total Injuries,870,908,703
3,Washington,Total Crashes,322,349,262,Total Fatalities,5,5,3,Total Injuries,199,225,172


In [21]:
fl_df_melt_2016 = pd.melt(fl_df_2014_2016,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2016 = fl_df_melt_2016.drop(['variable'],axis=1)
fl_df_melt_2016 = fl_df_melt_2016[['value',9, 16,2]]
fl_df_melt_2016.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2016.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,46,"4,398","5,980"
1,Baker,13,352,381
2,Bay,38,"2,290","4,089"
3,Bradford,6,243,362
4,Brevard,98,"6,564","8,752"
5,Broward,245,"25,361","41,755"
6,Calhoun,3,120,194
7,Charlotte,30,"1,649","2,721"
8,Citrus,24,"1,232","1,682"
9,Clay,26,"1,857","3,053"


In [22]:
df_combined_fl_16 = pd.merge(fl_df_melt_2016,pop_combined[['County', 2016]],on="County",how='left')
df_combined_fl_2016 = pd.merge(df_combined_fl_16,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2016.head()

,County,fatalaties,injuries,total_crashes,2016,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,46,"4,398","5,980",263959.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,13,352,381,27884.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,38,"2,290","4,089",183634.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,6,243,362,26740.0,NaN,NaN
4,Brevard,98,"6,564","8,752",576874.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [23]:
fl_df_melt_2015 = pd.melt(fl_df_2014_2016,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2015 = fl_df_melt_2015.drop(['variable'],axis=1)
fl_df_melt_2015 = fl_df_melt_2015[['value',10,17,3]]
fl_df_melt_2015.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2015.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,37,"3,826","5,027"
1,Baker,12,286,373
2,Bay,32,"2,317","3,946"
3,Bradford,10,241,348
4,Brevard,82,"6,130","8,387"
5,Broward,221,"23,473","38,409"
6,Calhoun,4,118,180
7,Charlotte,23,"1,616","2,556"
8,Citrus,29,"1,124","1,502"
9,Clay,33,"1,727","2,817"


In [24]:
df_combined_fl_15 = pd.merge(fl_df_melt_2015,pop_combined[['County', 2015]],on="County",how='left')
df_combined_fl_2015 = pd.merge(df_combined_fl_15,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2015.head()

,County,fatalaties,injuries,total_crashes,2015,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,37,"3,826","5,027",259052.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,12,286,373,27355.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,317","3,946",181488.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,241,348,26748.0,NaN,NaN
4,Brevard,82,"6,130","8,387",565746.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [25]:
fl_df_melt_2014 = pd.melt(fl_df_2014_2016,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2014 = fl_df_melt_2014.drop(['variable'],axis=1)
fl_df_melt_2014 = fl_df_melt_2014[['value',11,18,4]]
fl_df_melt_2014.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2014.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,30,"3,113","4,193"
1,Baker,2,264,246
2,Bay,30,"2,023","3,337"
3,Bradford,4,219,274
4,Brevard,65,"5,386","7,661"
5,Broward,173,"22,154","34,833"
6,Calhoun,2,94,134
7,Charlotte,19,"1,290","2,241"
8,Citrus,26,"1,099","1,471"
9,Clay,18,"1,784","2,594"


In [26]:
df_combined_fl_14 = pd.merge(fl_df_melt_2014,pop_combined[['County', 2014]],on="County",how='left')
df_combined_fl_2014 = pd.merge(df_combined_fl_14,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2014.head()

,County,fatalaties,injuries,total_crashes,2014,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,30,"3,113","4,193",255456.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,2,264,246,27122.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,30,"2,023","3,337",178289.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,4,219,274,26552.0,NaN,NaN
4,Brevard,65,"5,386","7,661",555548.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [27]:
df_combined_fl_2014.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2014_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2015.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2015_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2016.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2016_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2017

In [449]:
file_2017 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2017.pdf'

In [512]:
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[35].extract_text()
txt = txt[203:]
txt = txt[:-26]
txt = txt.split("\n")

In [520]:
new_list = []
for i in txt:
    my_dict = {}
    print(i)
    if i.startswith('Total Crashes'):
        crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",i)
        my_dict['total_crashes'] = crashes
    elif i.startswith('Total Fatalities'):
          fatalaties = re.findall("(?<=Total Fatalities)\s\d+\,?\d+",i)
          my_dict['fatalaties'] = fatalaties
    elif i.startswith('Total Injuries'):
          injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",i)
          my_dict['injuries'] = injuries
    elif any(i):
         county = i 
         my_dict['County'] = county  
    else:
        continue
#     my_dict['County'] = county
#     my_dict['fatalaties'] = fatalaties
#     my_dict['injuries'] = injuries
#     my_dict['total_crashes'] = crashes
    new_list.append(my_dict)
    print(my_dict)      
      

Total Crashes 6,075 5,980 5,027 1.59% 5,694
{'total_crashes': [' 6,075']}
Total Fatalities 55 46 37 19.57% 46
{'fatalaties': [' 55']}
Alachua
{'County': 'Alachua'}
Total Injuries 4,657 4,398 3,826 5.89% 4,294
{'injuries': [' 4,657']}
Total Crashes 431 381 373 13.12% 395
{'total_crashes': [' 431']}
Total Fatalities 7 13 12 -46.15% 11
{'fatalaties': []}
Baker
{'County': 'Baker'}
Total Injuries 321 352 286 -8.81% 320
{'injuries': [' 321']}
Total Crashes 4,051 4,089 3,946 -0.93% 4,029
{'total_crashes': [' 4,051']}
Total Fatalities 29 38 32 -23.68% 33
{'fatalaties': [' 29']}
Bay
{'County': 'Bay'}
Total Injuries 2,192 2,290 2,317 -4.28% 2,266
{'injuries': [' 2,192']}
Total Crashes 381 362 348 5.25% 364
{'total_crashes': [' 381']}
Total Fatalities 7 6 10 16.67% 8
{'fatalaties': []}
Bradford
{'County': 'Bradford'}
Total Injuries 250 243 241 2.88% 245
{'injuries': [' 250']}
Total Crashes 9,056 8,752 8,387 3.47% 8,732
{'total_crashes': [' 9,056']}
Total Fatalities 84 98 82 -14.29% 88
{'fatalatie

In [521]:
new_list

[{'total_crashes': [' 6,075']},
 {'fatalaties': [' 55']},
 {'County': 'Alachua'},
 {'injuries': [' 4,657']},
 {'total_crashes': [' 431']},
 {'fatalaties': []},
 {'County': 'Baker'},
 {'injuries': [' 321']},
 {'total_crashes': [' 4,051']},
 {'fatalaties': [' 29']},
 {'County': 'Bay'},
 {'injuries': [' 2,192']},
 {'total_crashes': [' 381']},
 {'fatalaties': []},
 {'County': 'Bradford'},
 {'injuries': [' 250']},
 {'total_crashes': [' 9,056']},
 {'fatalaties': [' 84']},
 {'County': 'Brevard'},
 {'injuries': [' 6,505']},
 {'total_crashes': [' 41,337']},
 {'fatalaties': [' 225']},
 {'County': 'Broward'},
 {'injuries': [' 24,762']},
 {'total_crashes': [' 207']},
 {'fatalaties': []},
 {'County': 'Calhoun'},
 {'injuries': [' 126']},
 {'total_crashes': [' 2,780']},
 {'fatalaties': [' 27']},
 {'County': 'Charlotte'},
 {'injuries': [' 1,594']},
 {'total_crashes': [' 1,781']},
 {'fatalaties': [' 31']},
 {'County': 'Citrus'},
 {'injuries': [' 1,382']},
 {'total_crashes': [' 2,961']},
 {'County': 'Cl

In [522]:
df = pd.DataFrame(new_list)
# df = df.drop([3,4,5,6], axis=1)
df

,total_crashes,fatalaties,County,injuries
0,"[ 6,075]",NaN,NaN,NaN
1,NaN,[ 55],NaN,NaN
2,NaN,NaN,Alachua,NaN
3,NaN,NaN,NaN,"[ 4,657]"
4,[ 431],NaN,NaN,NaN
5,NaN,[],NaN,NaN
6,NaN,NaN,Baker,NaN
7,NaN,NaN,NaN,[ 321]
8,"[ 4,051]",NaN,NaN,NaN
9,NaN,[ 29],NaN,NaN


In [435]:
new_list =[]
pages = list(range(35,42))
pdf = pdfplumber.open(file_2017)
for page in pages:    
    table= pdf.pages[page].extract_table()
    table = table[1:]
    t = {}
    t = table
    print(t)
#     df = pd.DataFrame(table)
    new_list.append(t)

[['Alachua', 'Total Crashes', '6,075', '5,980', '5,027', '1.59%\n19.57%\n5.89%', '5,694\n46\n4,294'], [None, 'Total Fatalities', '55', '46', '37', None, None], [None, 'Total Injuries', '4,657', '4,398', '3,826', None, None], [None, '', '', '', '', '', ''], ['Baker', 'Total Crashes', '431', '381', '373', '13.12%\n-46.15%\n-8.81%', '395\n11\n320'], [None, 'Total Fatalities', '7', '13', '12', None, None], [None, 'Total Injuries', '321', '352', '286', None, None], [None, '', '', '', '', '', ''], ['Bay', 'Total Crashes', '4,051', '4,089', '3,946', '-0.93%\n-23.68%\n-4.28%', '4,029\n33\n2,266'], [None, 'Total Fatalities', '29', '38', '32', None, None], [None, 'Total Injuries', '2,192', '2,290', '2,317', None, None], [None, '', '', '', '', '', ''], ['Bradford', 'Total Crashes', '381', '362', '348', '5.25%\n16.67%\n2.88%', '364\n8\n245'], [None, 'Total Fatalities', '7', '6', '10', None, None], [None, 'Total Injuries', '250', '243', '241', None, None], [None, '', '', '', '', '', ''], ['Brevard'

[['Volusia', 'Total Fatalities', '130', '122', '87', '6.56%', '113'], [None, 'Total Injuries', '6,527', '5,872', '5,750', '11.15%', '6,050'], [None, '', '', '', '', '', ''], ['Wakulla', 'Total Crashes', '432', '366', '348', '18.03%', '382'], [None, 'Total Fatalities', '7', '14', '2', '-50.00%', '8'], [None, 'Total Injuries', '302', '264', '261', '14.39%', '276'], [None, '', '', '', '', '', ''], ['Walton', 'Total Crashes', '1,300', '1,305', '1,248', '-0.38%\n23.53%\n-14.02%', '1,284\n16\n842'], [None, 'Total Fatalities', '21', '17', '9', None, None], [None, 'Total Injuries', '748', '870', '908', None, None], [None, '', '', '', '', '', ''], ['Washington', 'Total Crashes', '358', '322', '349', '11.18%\n100.00%\n-7.04%', '343\n7\n203'], [None, 'Total Fatalities', '10', '5', '5', None, None], [None, 'Total Injuries', '185', '199', '225', None, None], [None, '', '', '', '', '', '']]


In [434]:
new_list[1]

[['Clay', 'Total Injuries', '1,804', '1,857', '1,727', '-2.85%', '1,796'],
 [None, '', '', '', '', '', ''],
 ['Collier',
  'Total Crashes',
  '5,135',
  '5,273',
  '4,632',
  '-2.62%\n-34.29%\n-2.29%',
  '5,013\n36\n2,779'],
 [None, 'Total Fatalities', '23', '35', '49', None, None],
 [None, 'Total Injuries', '2,819', '2,885', '2,633', None, None],
 [None, '', '', '', '', '', ''],
 ['Columbia',
  'Total Crashes',
  '1,490',
  '1,527',
  '1,362',
  '-2.42%\n0.00%\n-3.87%',
  '1,460\n21\n1,121'],
 [None, 'Total Fatalities', '20', '20', '22', None, None],
 [None, 'Total Injuries', '1,117', '1,162', '1,084', None, None],
 [None, '', '', '', '', '', ''],
 ['DeSoto',
  'Total Crashes',
  '516',
  '453',
  '408',
  '13.91%\n36.36%\n28.24%',
  '459\n14\n337'],
 [None, 'Total Fatalities', '15', '11', '16', None, None],
 [None, 'Total Injuries', '386', '301', '323', None, None],
 [None, '', '', '', '', '', ''],
 ['Dixie',
  'Total Crashes',
  '151',
  '160',
  '159',
  '-5.63%\n100.00%\n25.00%',


In [429]:
d = pd.DataFrame(new_list)

,0,1,2,3,4,5,6
0,"[Alachua, Total Crashes, 6,075, 5,980, 5,027, ...","[Clay, Total Injuries, 1,804, 1,857, 1,727, -2...","[Glades, Total Fatalities, 13, 2, 14, 550.00%\...","[Jefferson, Total Crashes, 264, 320, 286, -17....","[Martin, , , , , , ]","[Polk, Total Injuries, 7,555, 6,804, 7,301, 11...","[Volusia, Total Fatalities, 130, 122, 87, 6.56..."
1,"[None, Total Fatalities, 55, 46, 37, None, None]","[None, , , , , , ]","[None, Total Injuries, 153, 97, 114, None, None]","[None, Total Fatalities, 4, 7, 5, -42.86%\n-15...","[Miami-Dade, Total Crashes, 65,986, 64,070, 63...","[None, , , , , , ]","[None, Total Injuries, 6,527, 5,872, 5,750, 11..."
2,"[None, Total Injuries, 4,657, 4,398, 3,826, No...","[Collier, Total Crashes, 5,135, 5,273, 4,632, ...","[None, , , , , , ]","[None, Total Injuries, 133, 157, 139, None, None]","[None, Total Fatalities, 285, 294, 339, -3.06%...","[Putnam, Total Crashes, 1,210, 1,157, 1,070, 4...","[None, , , , , , ]"
3,"[None, , , , , , ]","[None, Total Fatalities, 23, 35, 49, None, None]","[Gulf, Total Crashes, 183, 181, 140, 1.10%\n80...","[None, , , , , , ]","[None, Total Injuries, 32,389, 33,400, 32,742,...","[None, Total Fatalities, 26, 26, 20, 0.00%\n7....","[Wakulla, Total Crashes, 432, 366, 348, 18.03%..."
4,"[Baker, Total Crashes, 431, 381, 373, 13.12%\n...","[None, Total Injuries, 2,819, 2,885, 2,633, No...","[None, Total Fatalities, 9, 1, 2, None, None]","[Lafayette, Total Crashes, 35, 57, 42, -38.60%...","[None, , , , , , ]","[None, Total Injuries, 902, 843, 824, None, None]","[None, Total Fatalities, 7, 14, 2, -50.00%, 8]"
5,"[None, Total Fatalities, 7, 13, 12, None, None]","[None, , , , , , ]","[None, Total Injuries, 97, 119, 81, None, None]","[None, Total Fatalities, 5, 2, 4, None, None]","[Monroe, Total Crashes, 1,255, 2,391, 2,204, -...","[None, , , , , , ]","[None, Total Injuries, 302, 264, 261, 14.39%, ..."
6,"[None, Total Injuries, 321, 352, 286, None, None]","[Columbia, Total Crashes, 1,490, 1,527, 1,362,...","[None, , , , , , ]","[None, Total Injuries, 39, 37, 37, None, None]","[None, Total Fatalities, 18, 32, 21, None, None]","[Santa Rosa, Total Crashes, 1,893, 1,984, 1,94...","[None, , , , , , ]"
7,"[None, , , , , , ]","[None, Total Fatalities, 20, 20, 22, None, None]","[Hamilton, Total Crashes, 321, 269, 253, 19.33...","[None, , , , , , ]","[None, Total Injuries, 650, 1,279, 1,252, None...","[None, Total Fatalities, 19, 13, 19, None, None]","[Walton, Total Crashes, 1,300, 1,305, 1,248, -..."
8,"[Bay, Total Crashes, 4,051, 4,089, 3,946, -0.9...","[None, Total Injuries, 1,117, 1,162, 1,084, No...","[None, Total Fatalities, 9, 5, 5, None, None]","[Lake, Total Crashes, 4,731, 4,339, 3,918, 9.0...","[None, , , , , , ]","[None, Total Injuries, 1,597, 1,490, 1,664, No...","[None, Total Fatalities, 21, 17, 9, None, None]"
9,"[None, Total Fatalities, 29, 38, 32, None, None]","[None, , , , , , ]","[None, Total Injuries, 211, 159, 151, None, None]","[None, Total Fatalities, 48, 55, 61, None, None]","[Nassau, Total Crashes, 1,129, 1,116, 1,013, 1...","[None, , , , , , ]","[None, Total Injuries, 748, 870, 908, None, None]"


In [393]:
fl_df = pd.DataFrame()
pages = list(range(35,42))
pdf = pdfplumber.open(file_2017)
for page in pages:    
    table= pdf.pages[page].extract_table()
    table = table[1:]
    df = pd.DataFrame(table)
    fl_df = fl_df.append(df)

In [394]:
fl_df = fl_df.drop([3,4,5,6], axis=1)
fl_df

,0,1,2
0,Alachua,Total Crashes,"6,075"
1,None,Total Fatalities,55
2,None,Total Injuries,"4,657"
3,None,,
4,Baker,Total Crashes,431
5,None,Total Fatalities,7
6,None,Total Injuries,321
7,None,,
8,Bay,Total Crashes,"4,051"
9,None,Total Fatalities,29


In [408]:
stacked = fl_df.stack()
stacked

0   0             Alachua
    1       Total Crashes
    2               6,075
1   1    Total Fatalities
    2                  55
               ...       
12  2                  10
13  1      Total Injuries
    2                 185
14  1                    
    2                    
Length: 606, dtype: object

In [422]:
df = pd.DataFrame(stacked)
df.head(10)

0
0 0           Alachua
  1     Total Crashes
  2             6,075
1 1  Total Fatalities
  2                55
2 1    Total Injuries
  2             4,657
3 1                  
  2                  
4 0             Baker

In [424]:
df_new = df.dropna(axis=0,inplace = False)
df_new.head(20)

0
0 0           Alachua
  1     Total Crashes
  2             6,075
1 1  Total Fatalities
  2                55
2 1    Total Injuries
  2             4,657
3 1                  
  2                  
4 0             Baker
  1     Total Crashes
  2               431
5 1  Total Fatalities
  2                 7
6 1    Total Injuries
  2               321
7 1                  
  2                  
8 0               Bay
  1     Total Crashes

In [421]:
df_new.T

AttributeError: 'NoneType' object has no attribute 'T'

### Extracting data for years 2018 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [50]:
file_2018_2020 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2020.pdf'

In [54]:
fl_df_2018_2020 = pd.DataFrame()
pages = list(range(38,46))
pdf = pdfplumber.open(file_2018_2020)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2018_2020 = fl_df_2018_2020.append(df)

In [55]:
fl_df_2018_2020 = fl_df_2018_2020.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2018_2020 = fl_df_2018_2020.iloc[:-1 , :]
fl_df_2018_2020.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"4,617","5,878","5,846",Total Fatalities,48,72,51,Total Injuries,"3,266","4,237","4,415"
1,Baker,Total Crashes,420,438,400,Total Fatalities,4,7,5,Total Injuries,293,341,354
2,Bay,Total Crashes,"3,352","4,862","4,549",Total Fatalities,35,32,26,Total Injuries,"1,869","2,498","2,360"
3,Bradford,Total Crashes,329,412,449,Total Fatalities,10,12,9,Total Injuries,192,282,252
4,Brevard,Total Crashes,"8,395","9,207","9,233",Total Fatalities,81,84,85,Total Injuries,"5,768","6,625","6,911"


In [56]:
fl_df_2018_2020.tail()

,0,1,2,3,4,8,9,10,11,15,16,17,18
8,Union,Total Crashes,144,152,161,Total Fatalities,2,3,3,Total Injuries,135,117,142
0,Volusia,Total Crashes,"9,141","10,133","9,967",Total Fatalities,137,126,106,Total Injuries,"6,271","6,817","6,657"
1,Wakulla,Total Crashes,407,412,422,Total Fatalities,3,8,9,Total Injuries,215,262,301
2,Walton,Total Crashes,"1,342","1,383","1,412",Total Fatalities,29,19,24,Total Injuries,703,836,822
3,Washington,Total Crashes,432,360,348,Total Fatalities,10,9,10,Total Injuries,186,227,203


In [57]:
fl_df_melt_2020 = pd.melt(fl_df_2018_2020,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2020 = fl_df_melt_2020.drop(['variable'],axis=1)
fl_df_melt_2020 = fl_df_melt_2020[['value',9, 16,2]]
fl_df_melt_2020.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2020.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,48,"3,266","4,617"
1,Baker,4,293,420
2,Bay,35,"1,869","3,352"
3,Bradford,10,192,329
4,Brevard,81,"5,768","8,395"
5,Broward,268,"18,984","34,050"
6,Calhoun,3,120,198
7,Charlotte,26,"1,369","2,185"
8,Citrus,38,"1,194","1,685"
9,Clay,24,"1,502","2,511"


In [58]:
df_combined_fl_20 = pd.merge(fl_df_melt_2020,pop_combined[['County', 2020]],on="County",how='left')
df_combined_fl_2020 = pd.merge(df_combined_fl_20,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2020.head()

,County,fatalaties,injuries,total_crashes,2020,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,48,"3,266","4,617",278684.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,4,293,420,28309.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,35,"1,869","3,352",174461.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,192,329,28364.0,NaN,NaN
4,Brevard,81,"5,768","8,395",608007.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [59]:
fl_df_melt_2019 = pd.melt(fl_df_2018_2020,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2019 = fl_df_melt_2019.drop(['variable'],axis=1)
fl_df_melt_2019 = fl_df_melt_2019[['value',10,17,3]]
fl_df_melt_2019.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2019.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,72,"4,237","5,878"
1,Baker,7,341,438
2,Bay,32,"2,498","4,862"
3,Bradford,12,282,412
4,Brevard,84,"6,625","9,207"
5,Broward,214,"23,671","41,114"
6,Calhoun,4,148,213
7,Charlotte,22,"1,418","2,386"
8,Citrus,22,"1,410","1,774"
9,Clay,26,"1,669","2,694"


In [60]:
df_combined_fl_19 = pd.merge(fl_df_melt_2019,pop_combined[['County', 2019]],on="County",how='left')
df_combined_fl_2019 = pd.merge(df_combined_fl_19,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2019.head()

,County,fatalaties,injuries,total_crashes,2019,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,72,"4,237","5,878",269043.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,7,341,438,29210.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,498","4,862",174705.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,12,282,412,28201.0,NaN,NaN
4,Brevard,84,"6,625","9,207",601942.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [61]:
fl_df_melt_2018 = pd.melt(fl_df_2018_2020,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2018 = fl_df_melt_2018.drop(['variable'],axis=1)
fl_df_melt_2018 = fl_df_melt_2018[['value',11,18,4]]
fl_df_melt_2018.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2018.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,51,"4,415","5,846"
1,Baker,5,354,400
2,Bay,26,"2,360","4,549"
3,Bradford,9,252,449
4,Brevard,85,"6,911","9,233"
5,Broward,237,"24,644","41,082"
6,Calhoun,8,112,206
7,Charlotte,27,"1,582","2,734"
8,Citrus,37,"1,413","1,813"
9,Clay,19,"1,737","2,849"


In [62]:
df_combined_fl_18 = pd.merge(fl_df_melt_2018,pop_combined[['County', 2018]],on="County",how='left')
df_combined_fl_2018 = pd.merge(df_combined_fl_18,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2018.head()

,County,fatalaties,injuries,total_crashes,2018,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,51,"4,415","5,846",268851.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,5,354,400,28353.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,26,"2,360","4,549",186240.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,9,252,449,27752.0,NaN,NaN
4,Brevard,85,"6,911","9,233",595203.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [63]:
df_combined_fl_2018.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2018_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2019.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2019_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2020.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2020_FL.xlsx', engine='xlsxwriter',index=False)